In [1]:
# !pip install langchain-huggingface
# !pip install langchain-chroma

In [2]:
from langchain_community.document_loaders import PyPDFLoader,UnstructuredCSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import HypotheticalDocumentEmbedder
from langchain_chroma import Chroma
import pandas as pd
import os

In [3]:

os.environ['HUGGINGFACEHUB_API_TOKEN']= os.environ['HUGGINGFACEHUB_API_TOKEN']

In [4]:
hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "sentence-transformers/all-MiniLM-L6-v2",
    task="feature-extraction",
    huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
)


In [5]:


Llama3LLM = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)
Llama3LLMHydeEmbeddings = HypotheticalDocumentEmbedder.from_llm(Llama3LLM,
                                                  hf_embeddings,
                                                  prompt_key = "web_search")


MistralLLM = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)
MistralLLMHydeEmbeddings = HypotheticalDocumentEmbedder.from_llm(MistralLLM,
                                                  hf_embeddings,
                                                  prompt_key = "web_search")


GemmaLLM = HuggingFaceEndpoint(
    repo_id="google/gemma-7b",
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)
GemmaLLMHydeEmbeddings = HypotheticalDocumentEmbedder.from_llm(GemmaLLM,
                                                  hf_embeddings,
                                                  prompt_key = "web_search")

Llama2LLM = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-2-7b-hf",
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)
Llama2LLMHydeEmbeddings = HypotheticalDocumentEmbedder.from_llm(Llama2LLM,
                                                  hf_embeddings,
                                                  prompt_key = "web_search")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\91845\.cache\huggingface\token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\91845\.cache\huggingface\token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\9184

In [6]:
loader = UnstructuredCSVLoader(r"docs\Lex\podcastdata_dataset.csv")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
text_splitter.split_documents(docs)
documents=text_splitter.split_documents(docs)


In [7]:
## Design ChatPrompt Template
prompt = ChatPromptTemplate.from_template("""
just give main Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")


In [9]:
Llama2LLMDB=Chroma.from_documents(documents[:10000],Llama2LLMHydeEmbeddings)
Llama3LLMDB=Chroma.from_documents(documents[:10000],Llama3LLMHydeEmbeddings)
MistralLLMDB = Chroma.from_documents(documents[:10000],MistralLLMHydeEmbeddings)
GemmaLLMDb=Chroma.from_documents(documents[:10000],GemmaLLMHydeEmbeddings)

In [10]:
Llama2LLMRetriever = Llama2LLMDB.as_retriever(type="similarity")
Llama3LLMRetriever = Llama3LLMDB.as_retriever(type="similarity")
MistralLLMRetriever = MistralLLMDB.as_retriever(type="similarity")
GemmaLLMRetriever = GemmaLLMDb.as_retriever(type="similarity")


In [11]:
# Create Stuff Docment Chain for Llama3
DocumentChainLlama3LLM = create_stuff_documents_chain(Llama3LLM,prompt)
RetrievalChainLlama3LLM = create_retrieval_chain(Llama3LLMRetriever,DocumentChainLlama3LLM)

## Create Stuff Docment Chain for Llama2
DocumentChainLlama2LLM = create_stuff_documents_chain(Llama2LLM,prompt)
RetrievalChainLlama2LLM = create_retrieval_chain(Llama2LLMRetriever,DocumentChainLlama2LLM)


# ## Create Stuff Docment Chain for Gemma
DocumentChainGemmaLLM = create_stuff_documents_chain(GemmaLLM,prompt)
RetrievalChainGemmaLLM = create_retrieval_chain(GemmaLLMRetriever,DocumentChainGemmaLLM)


## Create Stuff Docment Chain for mistral
DocumentChainMistralLLM = create_stuff_documents_chain(MistralLLM,prompt)
RetrievalChainMistralLLM = create_retrieval_chain(MistralLLMRetriever,DocumentChainMistralLLM)


In [13]:
if __name__ == "__main__":
    MistralOutput = RetrievalChainMistralLLM.invoke({"input":"How does Lex Fridman approach philosophical questions in his discussions?"})
    # Llama2Output = RetrievalChainLlama2LLM.invoke({"input":"How does Lex Fridman approach philosophical questions in his discussions?"})
    GemmaOutput = RetrievalChainGemmaLLM.invoke({"input":"How does Lex Fridman approach philosophical questions in his discussions?"})
    Llama3Output = RetrievalChainLlama3LLM.invoke({"input":"How does Lex Fridman approach philosophical questions in his discussions?"})

In [ ]:
# print(MistralOutput['answer'])

## Comparing diffrent LLMS

In [14]:
from langchain.evaluation import load_evaluator


custom_criteria = {
    "simplicity": "Is the language straightforward and unpretentious?",
    "clarity": "Are the sentences clear and easy to understand?",
    "precision": "Is the writing precise, with no unnecessary words or details?",
    "truthfulness": "Does the writing feel honest and sincere?",
    "subtext": "Does the writing suggest deeper meanings or themes?",
}
evaluator = load_evaluator("pairwise_string", criteria=custom_criteria, llm=MistralLLM)

This chain was only tested with GPT-4. Performance may be significantly worse with other models.


In [15]:
questions = [
"What does Lex Fridman discuss about the ethical implications of AI?",
"What insights are shared on human-robot interaction?",
"What are some technological innovations mentioned in the podcast?",
"How does Lex Fridman approach philosophical questions in his discussions?",
"What expertise do guests like Elon Musk bring to the podcast?"
]

In [16]:
evaluator.evaluate_string_pairs(
    prediction=f"{MistralOutput['answer']}",
    prediction_b=f"{GemmaOutput['answer']}",
    input={questions[0]},
)

{'reasoning': '\n\nEvaluation:\n- Simplicity: Both assistants use simple language.\n- Clarity: Both assistants provide clear answers.\n- Precision: Both assistants are precise.\n- Truthfulness: Both assistants answer truthfully.\n- Subtext: Assistant B suggests a step-by-step approach that Lex Fridman follows to discuss philosophical questions.\n\nVerdict: [[B]] Assistant B provides a more detailed explanation of how Lex Fridman approaches philosophical questions.',
 'value': 'B',
 'score': 0}

In [17]:
evaluator.evaluate_string_pairs(
    prediction=f"{GemmaOutput['answer']}",
    prediction_b=f"{Llama3Output['answer']}",
    input={questions[0]},
)

{'reasoning': "\n\nEvaluation:\nAssistant A's response is clear, concise, and focused on the question asked by the user. It provides a direct answer without any unnecessary details, making it easy for the user to understand. Assistant A's response is also truthful and sincere, as it accurately describes Lex Fridman's approach to philosophical questions. Assistant B's response, while detailed, provides a more specific answer based on the context of Lex Fridman's discussion on Tesla cars. However, it does not directly answer the question asked by the user about the ethical implications of AI, which is the primary focus of the user's question. Therefore, Assistant A's answer is more relevant to the user's question and is more helpful.\n\nVerdict: [[A]]",
 'value': 'A',
 'score': 1}

---

## EVALUVATION

In [ ]:
questions = [
"What does Lex Fridman discuss about the ethical implications of AI?",
"What insights are shared on human-robot interaction?",
"What are some technological innovations mentioned in the podcast?",
"How does Lex Fridman approach philosophical questions in his discussions?",
"What expertise do guests like Elon Musk bring to the podcast?"
]

In [22]:

import os
import nest_asyncio
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langsmith import Client
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.smith import RunEvalConfig, run_on_dataset
import numpy as np

nest_asyncio.apply()

# --------------------------------------------------------------
# Load API Keys From the .env File
# --------------------------------------------------------------

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Document Q&A 2"


# --------------------------------------------------------------
# LangSmith Quick Start
# Load the LangSmith Client and Test Run
# --------------------------------------------------------------

client = Client()

dataset_name = "Q&A data 4"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Data for evaluvation",
)

for input_prompt in questions:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"input": input_prompt},
        outputs=None,
        dataset_id=dataset.id,
    )

In [23]:
# --------------------------------------------------------------
# 3. Evaluate Datasets Without Labels
# --------------------------------------------------------------

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria(
            {"creativity": "Is this submission creative, imaginative, or novel?",
                "simplicity": "Is the language straightforward and unpretentious?",
                "clarity": "Are the sentences clear and easy to understand?",
                "precision": "Is the writing precise, with no unnecessary words or details?",
                "truthfulness": "Does the writing feel honest and sincere?",
                "subtext": "Does the writing suggest deeper meanings or themes?"}
        ),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness")
    ],
    eval_llm=MistralLLM,
)

run_on_dataset(
    client=client,
    dataset_name="Q&A data 4",
    llm_or_chain_factory=RetrievalChainMistralLLM,
    evaluation=evaluation_config,

)

View the evaluation results for project 'crushing-cloth-55' at:
https://smith.langchain.com/o/38ca8262-a246-5483-880a-a4eae38bb4bc/datasets/a882df73-97b2-4922-83f9-c80d0a046681/compare?selectedSessions=4be610ff-9322-4e5a-ba15-9b6170af5ba6

View all tests for Dataset Q&A data 4 at:
https://smith.langchain.com/o/38ca8262-a246-5483-880a-a4eae38bb4bc/datasets/a882df73-97b2-4922-83f9-c80d0a046681
[------------------------------------------------->] 5/5

{'project_name': 'crushing-cloth-55',
 'results': {'158a56ab-11ab-4ad1-bb05-b03ad238e8c7': {'input': {'input': 'What expertise do guests like Elon Musk bring to the podcast?'},
   'feedback': [EvaluationResult(key='creativity-simplicity-clarity-precision-truthfulness-subtext', score=None, value=None, comment='Error evaluating run f444daae-6325-45b8-a3b3-d0acffeaac65: Could not map run prediction with multiple keys: {\'input\': \'What expertise do guests like Elon Musk bring to the podcast?\', \'context\': [Document(page_content="The following is a conversation with Elon Musk. He\'s the CEO of Tesla, SpaceX, Neuralink, and a cofounder of several other companies. This conversation is part of the Artificial Intelligence podcast. The series includes leading researchers in academia and industry, including CEOs and CTOs of", metadata={\'source\': \'docs\\\\Lex\\\\podcastdata_dataset.csv\'}), Document(page_content="The following is a conversation with Elon Musk. He\'s the CEO of Tesla, SpaceX

You can see Evaluvation at : [Langsmith](https://smith.langchain.com/public/b07c7bda-41e1-45fd-9a7c-3c9bebf40ac4/d)

---